In [54]:
import sys

sys.path.append('..')

import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import requests
import csv
from datetime import datetime

TODAY = datetime.today().strftime("%d/%m/%Y")

from utils import get_wallet
from prices import get_portfolio_prices

In [2]:
def mount_wallet_data_frame(google_sheet_key: str) -> pd.DataFrame:
    
    wallet_items = get_wallet(google_sheet_key)
    
    wallet = pd.DataFrame(wallet_items).set_index('Date')
    
    wallet.index = pd.to_datetime(wallet.index)
    
    return wallet

In [3]:
def get_assets_by_type(wallet, asset_type):
    
    asset_filter = wallet[wallet.asset_type == asset_type]
    
    assets = set(asset_filter.asset_name.to_list())
    
    return assets

In [4]:
def get_portfolio_performace(wallet):
    
    funds = get_assets_by_type(wallet, 'fund')
    stocks = get_assets_by_type(wallet, 'stock')
    etfs = get_assets_by_type(wallet, 'etf')
    
    allocations = wallet.pivot(columns = 'asset_name').value.fillna(0)
    
    portfolio_prices = get_portfolio_prices(stocks, funds, etfs, start_date='01/01/2018').asfreq('B')
    
    return portfolio_prices

In [5]:
def mount_wallet_shares(wallet, prices):
    
    wallet_shares = []
    
    for i in range(len(wallet)):
        
        asset_name = wallet.iloc[i].asset_name
        value = wallet.iloc[i].value
        date = wallet.index[i]
        
        shares = float(value) / prices[asset_name][date]
        
        wallet_shares.append(shares)
        
    wallet['shares'] = wallet_shares 
    
    for asset in wallet.asset_name.unique():
        
        wallet.loc[wallet.asset_name == asset, 'shares'] = wallet[wallet.asset_name == asset].shares.cumsum()
    
    return wallet

In [6]:
def get_allocation_time_series(wallet):
    
    allocations = wallet.pivot(columns = 'asset_name').shares.asfreq('B')
    
    last_date = allocations.index[-1]
    
    df = pd.DataFrame(pd.date_range(start=last_date, end='2021-04-07', freq='B'))
    df.columns = ['Date']
    
    allocations = allocations.merge(df, on='Date', how='outer').set_index('Date')
    
    allocations = allocations.fillna(method='ffill').astype(np.float64).asfreq('B')
    
    return allocations

In [7]:
def get_portfolio(prices, allocations):
    
    portfolio = (prices.loc[:, allocations.columns] * allocations).asfreq('B').fillna(method='ffill')
    
    return portfolio

def get_patrimony(portfolio):
    
    patrimony = portfolio.sum(axis=1)
    
    return patrimony

In [8]:
def get_returns(prices, portfolio):
    
    weights = portfolio.div(portfolio.sum(axis=1), axis=0).fillna(0)
    
    returns = pd.DataFrame(index = weights.index)
    returns['Returns'] = 0

    for time in weights.index:
        returns['Returns'].loc[time] = np.dot(prices.pct_change().loc[time], weights.loc[time])
        
    return returns

In [9]:
def get_cumulative_returns(prices, portfolio):
    
    returns = get_returns(prices, portfolio)
                          
    cumulative_returns = (1 + returns).cumprod()
                          
    return cumulative_returns

In [10]:
wallet = mount_wallet_data_frame('14KWHuqot8Fr74-Yw4x0brDDdSZDoQL7SBiN1Q5vL188')

In [11]:
portfolio_prices = get_portfolio_performace(wallet)
prices = portfolio_prices['Close']

In [12]:
shares = mount_wallet_shares(wallet, prices)

In [13]:
allocations = get_allocation_time_series(shares)

In [14]:
portfolio = get_portfolio(prices, allocations)

In [15]:
patrimony = get_patrimony(portfolio)

In [16]:
returns = get_returns(prices, portfolio)

In [17]:
cumulative_returns = get_cumulative_returns(prices, portfolio)

In [18]:
# Váriaveis finais cumulative_returns, returns, patrimony. Calcular métricas a partir da variável de returns

In [66]:
class Portfolio:
    
    def __init__(self, google_sheet_key: str):
        
        self.wallet = self._mount_wallet_data_frame('14KWHuqot8Fr74-Yw4x0brDDdSZDoQL7SBiN1Q5vL188')
        
        self.prices = self._get_portfolio_prices()
        
        self._mount_wallet_shares()
        
        self.allocations = self._get_allocation_time_series()
        
        self.portfolio = self._get_portfolio()
        
    def get_patrimony(self):

        patrimony = self.portfolio.sum(axis=1)

        return patrimony
    
    def get_returns(self):
    
        weights = self.portfolio.div(self.portfolio.sum(axis=1), axis=0).fillna(0)

        returns = pd.DataFrame(index = weights.index)
        returns['Returns'] = 0

        for time in weights.index:
            returns['Returns'].loc[time] = np.dot(self.prices['Close'].pct_change().loc[time], weights.loc[time])

        return returns
    
    def get_cumulative_returns(self):
    
        returns = get_returns(self.prices['Close'], self.portfolio)

        cumulative_returns = (1 + returns).cumprod()

        return cumulative_returns
        
    def _get_wallet(self, google_sheet_key: str) -> list: 
        """Gets wallet from google public spreadsheet
        Args
        - google_sheet_key (str): key placed on the url of the spreadsheet
        Returns
        - (list<dict>): each item represents a position on the wallet and the dict keys are: date, securitie and quantity

        >>> get_wallet('1KHC1z4eO7CSekLK0ZaypIzrO5c5pVCq8Gx-hxL-sUaM')
        >>> [{'data': '04/01/2021', 'ativo': 'ITSA4', 'quantidade': '200'},
             {'data': '11/01/2021', 'ativo': 'OIBR3', 'quantidade': '1000'},
             {'data': '22/07/2020', 'ativo': 'BPAC11', 'quantidade': '500'}]
        """
        google_sheet_url = f"https://docs.google.com/spreadsheet/ccc?key={google_sheet_key}&output=csv"
        r = requests.get(google_sheet_url) 
        
        assert r.status_code == 200 
        
        decoded_content = r.content.decode('utf-8') 
        reader = csv.DictReader(decoded_content.splitlines(), delimiter=',')
        
        return [row for row in reader]
    
    def _mount_wallet_data_frame(self, google_sheet_key: str) -> pd.DataFrame:
    
        wallet_items = self._get_wallet(google_sheet_key)

        wallet = pd.DataFrame(wallet_items).set_index('Date')

        wallet.index = pd.to_datetime(wallet.index)

        return wallet
    
    def _get_assets_by_type(self, wallet, asset_type):
    
        asset_filter = wallet[wallet.asset_type == asset_type]

        assets = set(asset_filter.asset_name.to_list())

        return assets
    
    def _get_portfolio_prices(self):
    
        funds = get_assets_by_type(self.wallet, 'fund')
        stocks = get_assets_by_type(self.wallet, 'stock')
        etfs = get_assets_by_type(self.wallet, 'etf')

        allocations = wallet.pivot(columns = 'asset_name').value.fillna(0)

        portfolio_prices = get_portfolio_prices(stocks, funds, etfs, start_date='01/01/2018').asfreq('B')

        return portfolio_prices
    
    def _mount_wallet_shares(self):

        wallet_shares = []

        for i in range(len(self.wallet)):

            asset_name = self.wallet.iloc[i].asset_name
            value = self.wallet.iloc[i].value
            date = self.wallet.index[i]

            shares = float(value) / self.prices['Close'][asset_name][date]

            wallet_shares.append(shares)

        self.wallet['shares'] = wallet_shares 

        for asset in self.wallet.asset_name.unique():

            self.wallet.loc[self.wallet.asset_name == asset, 'shares'] = self.wallet[self.wallet.asset_name == asset].shares.cumsum()
        
    def _get_allocation_time_series(self):
    
        allocations = self.wallet.pivot(columns = 'asset_name').shares.asfreq('B')

        last_date = allocations.index[-1]

        df = pd.DataFrame(pd.date_range(start=last_date, end=TODAY, freq='B'))
        df.columns = ['Date']

        allocations = allocations.merge(df, on='Date', how='outer').set_index('Date')

        allocations = allocations.fillna(method='ffill').astype(np.float64).asfreq('B')

        return allocations
    
    def _get_portfolio(self):
    
        portfolio = (self.prices['Close'].loc[:, self.allocations.columns] * self.allocations).asfreq('B').fillna(method='ffill')

        return portfolio



In [67]:
port = Portfolio('14KWHuqot8Fr74-Yw4x0brDDdSZDoQL7SBiN1Q5vL188')

In [62]:
port.get_cumulative_returns()

,Returns
Date,
2018-01-02,NaN
2018-01-03,1.000000
2018-01-04,1.000000
2018-01-05,1.000000
2018-01-08,1.000000
...,...
2021-04-14,0.959992
2021-04-15,0.944059
2021-04-16,0.950854


In [64]:
port.get_patrimony()

Date
2018-01-02       0.000000
2018-01-03       0.000000
2018-01-04       0.000000
2018-01-05       0.000000
2018-01-08       0.000000
                 ...     
2021-04-14    3555.452504
2021-04-15    3494.951031
2021-04-16    3519.198270
2021-04-19    3563.100319
2021-04-20    3565.999257
Freq: B, Length: 861, dtype: float64

In [68]:
port.get_returns()

,Returns
Date,
2018-01-02,NaN
2018-01-03,0.000000
2018-01-04,0.000000
2018-01-05,0.000000
2018-01-08,0.000000
...,...
2021-04-14,-0.003430
2021-04-15,-0.016598
2021-04-16,0.007199
